### Imports


In [ ]:
import cv2
import os
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Constants


In [ ]:
yolo_annotations_mask_path = "/content/drive/My Drive/Data for students/yolo_annotations_mask"  # Путь до файлов в формате YOLO для маскирования/сегментации
yolo_annotations_path = "/content/drive/My Drive/Data for students/yolo_annotations"            # Путь до файлов в формате YOLO для детекции
path_to_folder = "/content/drive/My Drive/Data for students/img"                                # Путь к исходным изображениям хлопьев
output_folder = "/content/drive/My Drive/Data for students/processed_img_bbox"                  # Путь для вывода изображений после разметки сегментации/детекции

### Crop image

In [ ]:
def crop_image(image, crop_top=0, crop_left=0, crop_right=0, crop_bottom=0):
    """
    Функция для обрезки изображения с указанием процента обрезки по каждому краю.

    Параметры:
    - image: ndarray
        Исходное изображение в формате NumPy массива (например, загруженное с помощью OpenCV).
    - crop_top: float, optional (default=0)
        Процент обрезки сверху от общей высоты изображения.
    - crop_left: float, optional (default=0)
        Процент обрезки слева от общей ширины изображения.
    - crop_right: float, optional (default=0)
        Процент обрезки справа от общей ширины изображения.
    - crop_bottom: float, optional (default=0)
        Процент обрезки снизу от общей высоты изображения.

    Возвращает:
    - cropped_image: ndarray
        Обрезанное изображение в виде копии исходного массива.
    """
    height, width = image.shape[:2]

    top_px = int(height * crop_top / 100)
    left_px = int(width * crop_left / 100)
    right_px = int(width * crop_right / 100)
    bottom_px = int(height * crop_bottom / 100)

    # if top_px + bottom_px >= height:
    #     raise ValueError("Сумма процентов обрезки сверху и снизу должна быть меньше 100% высоты изображения.")
    # if left_px + right_px >= width:
    #     raise ValueError("Сумма процентов обрезки слева и справа должна быть меньше 100% ширины изображения.")

    cropped_image = image[top_px:height - bottom_px, left_px:width - right_px]

    return cropped_image.copy()

#### Image loader

In [ ]:
def load_images_from_folder(folder_path):
    """
    Загружает все изображения из указанной папки и возвращает их в виде словаря.

    Параметры:
    - folder_path (str): Путь к папке с изображениями.

    Возвращает:
    - dict: Словарь, где ключи — имена файлов изображений, а значения — загруженные кадры.
    """
    images = {}
    # Расширения файлов, которые считаются изображениями
    supported_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')

    # Проверяем, существует ли папка
    if not os.path.isdir(folder_path):
        raise ValueError(f"Папка по пути '{folder_path}' не найдена.")

    # Проходим по всем файлам в папке
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            img_path = os.path.join(folder_path, filename)
            # Читаем изображение с помощью OpenCV
            img = cv2.imread(img_path)
            if img is not None:
                images[filename] = img
            else:
                print(f"Предупреждение: Не удалось загрузить изображение '{filename}'. Оно будет пропущено.")

    return images

### To YOLO format

In [ ]:
import functools as f

def mask_to_yolo_format(image, contours, classes):
    """
    Преобразует контуры объектов изображения в формат аннотаций YOLO.

    Параметры:
    - image: ndarray
        Изображение, к которому применяются контуры, в формате NumPy массива (например, загруженное с помощью OpenCV).
        Требуется, чтобы изображение имело форму (height, width, channels).
    - contours: list
        Список контуров объектов на изображении. Каждый контур — это список точек,
        где каждая точка представлена в формате [[x, y]].
    - classes: list
        Список классов, соответствующих каждому контуру. Каждый элемент — это ID класса (целое число).

    Возвращает:
    - yolo_annotations: list
        Список строк аннотаций в формате YOLO. Каждая строка содержит:
        - ID класса,
        - координаты всех точек контура, нормализованные в диапазон [0, 1] относительно ширины и высоты изображения.
    """
    height, width, _ = image.shape

    yolo_annotations = []

    for contour, class_id in zip(contours, classes):
        xys = [[point[0][0] / width, point[0][1] / height] for point in contour]
        to_str = f.reduce(lambda xs, x: xs + x, xys)
        annotation = f"{class_id} " + " ".join(map(str, to_str))
        yolo_annotations.append(annotation)

    return yolo_annotations

In [ ]:
def convert_to_yolo_format(image, bboxes, class_ids):
    """
    Преобразует ограничивающие рамки объектов (bounding boxes) в формат аннотаций YOLO.

    Параметры:
    - image: ndarray
        Изображение, к которому применяются bounding boxes, в формате NumPy массива
        (например, загруженное с помощью OpenCV). Требуется, чтобы изображение имело форму (height, width, channels).
    - bboxes: list
        Список ограничивающих рамок объектов. Каждая рамка представлена в формате [x_min, y_min, x_max, y_max],
        где:
          * x_min, y_min — координаты верхнего левого угла рамки;
          * x_max, y_max — координаты нижнего правого угла рамки.
    - class_ids: list
        Список классов, соответствующих каждому bounding box. Каждый элемент — это ID класса (целое число).

    Возвращает:
    - yolo_annotations: list
        Список строк аннотаций в формате YOLO. Каждая строка содержит:
        - ID класса,
        - координаты центра рамки (x_center, y_center),
        - ширину и высоту рамки (bbox_width, bbox_height),
        нормализованные в диапазон [0, 1] относительно ширины и высоты изображения.
    """
    height, width, _ = image.shape

    yolo_annotations = []

    for bbox, class_id in zip(bboxes, class_ids):
        x_min, y_min, x_max, y_max = bbox

        # Вычисляем центр и размеры
        x_center = (x_min + x_max) / 2.0
        y_center = (y_min + y_max) / 2.0
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min

        # Нормализуем значения
        x_center /= width
        y_center /= height
        bbox_width /= width
        bbox_height /= height

        # Формируем строку в формате YOLO
        annotation = f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
        yolo_annotations.append(annotation)

    return yolo_annotations

### Find dominant color

In [ ]:
import matplotlib.pyplot as plt

def plot_kmeans_clusters(blue_red_pixels, labels, cluster_centers):
    """
    Строит визуализацию кластеров KMeans для двухмерных данных (синие и красные компоненты).

    Параметры:
    - blue_red_pixels: ndarray
        Двумерный массив пикселей, где каждая строка представляет собой пару значений
        (интенсивность синего и красного каналов) в формате [[B, R], ...].
    - labels: ndarray
        Одномерный массив меток кластеров, присвоенных каждому пикселю алгоритмом KMeans.
    - cluster_centers: ndarray
        Двумерный массив координат центров кластеров, вычисленных KMeans, в формате [[B, R], ...].

    Описание работы:
    1. Для каждого кластера определяются точки, принадлежащие этому кластеру.
    2. Точки каждого кластера отображаются на диаграмме рассеяния разными цветами.
    3. Центры кластеров отображаются черными крестиками (`x`) для визуального обозначения их положения.
    4. Диаграмма имеет подписи осей, заголовок, легенду и сетку для улучшения восприятия.
    """
    plt.figure(figsize=(10, 6))
    for cluster_id in np.unique(labels):
      cluster_points = blue_red_pixels[labels == cluster_id]
      plt.scatter(
          cluster_points[:, 0], cluster_points[:, 1],
          label=f"Cluster {cluster_id + 1}",
          s=10
      )
    plt.scatter(
        cluster_centers[:, 0], cluster_centers[:, 1],
        color='black',
        marker='x',
        s=200,
        label='Cluster Centers'
    )

    plt.title("KMeans Clustering on Blue-Red Pixels")
    plt.xlabel("Blue Channel Intensity")
    plt.ylabel("Red Channel Intensity")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
from sklearn.cluster import KMeans

def find_dominant_color(image, bbox, n_colors=2):
    """
    Определяет доминирующий цвет в заданной области изображения.

    Параметры:
    - image: ndarray
        Исходное изображение в формате NumPy массива (например, загруженное с помощью OpenCV).
    - bbox: tuple (x, y, w, h)
        Ограничивающая рамка (bounding box), задающая область интереса:
        * x, y — координаты верхнего левого угла рамки;
        * w, h — ширина и высота рамки.
    - n_colors: int, optional (default=2)
        Количество кластеров для метода KMeans (сколько основных цветов будет определено).

    Возвращает:
    - paint_color: tuple (B, G, R)
        Доминирующий цвет в области интереса в формате BGR:
        * (255, 0, 0) — если синий цвет преобладает;
        * (0, 0, 255) — если красный цвет преобладает.

    Описание работы:
    1. Из области изображения, заданной `bbox`, выделяется подмассив (область интереса).
    2. Удаляются белые пиксели (пиксели с компонентами R, G, B, равными 255), чтобы исключить фон.
    3. Выбираются только две цветовые компоненты: синий (B) и красный (R), игнорируя зеленый (G).
    4. Используется алгоритм KMeans для кластеризации пикселей по их цветам (синие и красные группы).
    5. Считаются доминирующие цвета, и выбирается тот, который преобладает.
    6. Возвращается `paint_color`, указывающий доминирующий цвет (синий или красный).
    """
    x, y, w, h = bbox

    # Вырезаем область пятна
    spot = image[y:y+h, x:x+w]
    # Удаление белых пикселей
    spot = spot[~((spot[:, :, 0] == 255) & (spot[:, :, 1] == 255) & (spot[:, :, 2] == 255))]

    spot = spot.reshape(-1, 3)
    spot = spot[:, [0, 2]] # Фичи: blue, red

    kmeans = KMeans(n_clusters=n_colors, max_iter=1000, random_state=0)
    kmeans.fit(spot)

    # plot_kmeans_clusters(spot, kmeans.labels_, kmeans.cluster_centers_)

    colors = kmeans.cluster_centers_.astype(int)
    counts = np.bincount(kmeans.labels_)

    sorted_indices = np.argsort(-counts)
    sorted_colors = colors[sorted_indices]

    dominant_color = tuple(map(int, sorted_colors[0]))

    if dominant_color[0] > dominant_color[1]:
        paint_color = (255, 0, 0) # blue
    else:
        paint_color = (0, 0, 255) # red

    return paint_color


### Find counters


In [ ]:
def add_white_border(image, border_size=10):
    """
    Добавляет белую рамку вокруг изображения.

    Параметры:
    - image: ndarray
        Исходное изображение.
    - border_size: int, optional (default=10)
        Размер белой рамки в пикселях с каждой стороны.

    Возвращает:
    - image_with_border: ndarray
        Изображение с добавленной белой рамкой.
    """
    white_color = (255, 255, 255)
    return cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=white_color)

def preprocess_image(image):
    """
    Выполняет предварительную обработку изображения для выделения контуров.

    Параметры:
    - image: ndarray
        Изображение в оттенках серого.

    Возвращает:
    - eroded: ndarray
        Обработанное изображение с готовыми для поиска контурами.
    """
    blurred = cv2.GaussianBlur(image, (11, 11), 1.4)
    edges = cv2.Canny(blurred, 12, 25)

    # Применяем морфологическое закрытие, чтобы заполнить дыры
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # **Расширение** для объединения близких контуров
    dilation_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    dilated = cv2.dilate(closed, dilation_kernel, iterations=1)

    # **Эрозия** для уточнения контуров после расширения
    erosion_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    eroded = cv2.erode(dilated, erosion_kernel, iterations=1)

    return eroded

def save_yolo_annotations(yolo_annotations, filename, output_path):
    """
    Сохраняет аннотации YOLO в текстовый файл.

    Параметры:
    - yolo_annotations: list
        Список строк аннотаций в формате YOLO.
    - filename: str
        Имя файла без расширения.
    - output_path: str
        Путь к директории, где будет сохранён файл.

    Возвращает:
    - None
    """
    file_path = f"{output_path}/{filename}.txt"
    with open(file_path, 'w') as file:
        file.write('\n'.join(yolo_annotations))

In [ ]:
def create_mask(image, contours, colors):
    """
    Создает маску, закрашивая контуры заданными цветами.

    Параметры:
    - image: ndarray
        Исходное изображение, определяющее размер маски.
    - contours: list
        Список контуров, которые необходимо закрасить. Каждый контур представлен массивом координат.
    - colors: list
        Список цветов для каждого контура. Каждый цвет — это кортеж в формате BGR, например, (255, 0, 0).

    Возвращает:
    - mask: ndarray
        Изображение-маска того же размера, что и `image`, с закрашенными контурами.

    Описание работы:
    1. Создается пустая белая маска того же размера, что и исходное изображение.
    2. Каждый контур из списка `contours` закрашивается соответствующим цветом из `colors`.
    3. Для закрашивания используется функция OpenCV `cv2.drawContours` с параметром `thickness=cv2.FILLED`.
    """
    # Создаем пустую маску того же размера, что и изображение
    mask = np.zeros(image.shape, dtype=np.uint8) + 255

    # Перекрашиваем каждый контур в свой цвет
    for contour, color in zip(contours, colors):
        cv2.drawContours(mask, [contour], -1, color, thickness=cv2.FILLED)

    return mask

In [ ]:
from google.colab.patches import cv2_imshow
import imutils

def my_find_contours(image, filename, display_steps=False):
    """
    Находит контуры на изображении, определяет доминирующие цвета внутри областей,
    формирует маску и аннотации в формате YOLO.

    Параметры:
    - image: ndarray
        Исходное изображение.
    - filename: str
        Имя файла для сохранения результата.
    - display_steps: bool, optional (default=False)
        Если True, промежуточные шаги обработки изображения будут визуализированы.

    Возвращает:
    - image: ndarray
        Изображение с обработанными контурами.
    """
    # Добавляем белую рамку
    image_with_border = add_white_border(image)

    # Конвертируем в оттенки серого
    if len(image_with_border.shape) == 3:
        gray = cv2.cvtColor(image_with_border, cv2.COLOR_BGR2GRAY)
    else:
        gray = image_with_border.copy()

    if display_steps:
        cv2_imshow(gray)

    # Обработка изображения
    eroded = preprocess_image(gray)
    eroded = eroded[10:-10, 10:-10]

    if display_steps:
        cv2_imshow(eroded)

    # Поиск контуров
    contours = cv2.findContours(eroded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    # Сегментация
    # for c in contours:
    #     # Вычисление периметра контура
    #     p = cv2.arcLength(c, True)
    #     # Аппроксимация контура
    #     approx = cv2.approxPolyDP(c, 0.02 * p, True)
    #     # Отрисовка контуров
    #     cv2.drawContours(image, [approx], -1, (0, 255, 0), 1)  # Зеленые контуры (BGR: 0, 255, 0)

    # Детекция
    bboxes = []  # Для хранения ограничивающих рамок
    classes = [] # Для хранения класса для каждой рамки (0 -- синий, 1 -- красный)
    colors = []  # Для хранения самих цветов
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        dominant_color = find_dominant_color(image, (x, y, w, h))
        bboxes.append((x, y, x + w, y + h))
        classes.append(0 if dominant_color[0] == 255 else 1)
        colors.append(dominant_color)

    # Создание маски
    # mask = create_mask(image, contours, colors)

    # Сохранение аннотаций YOLO
    yolo_annotations = mask_to_yolo_format(image, contours, classes)
    save_yolo_annotations(yolo_annotations, filename, yolo_annotations_mask_path)

    return image

In [ ]:
def my_find_contours_with_bbox(image, filename, display_steps=False):
    """
    Находит контуры на изображении, определяет доминирующие цвета и рисует bounding box для каждого объекта.

    Параметры:
    - image: ndarray
        Исходное изображение.
    - filename: str
        Имя файла для сохранения результата.
    - display_steps: bool, optional (default=False)
        Если True, промежуточные шаги обработки изображения будут визуализированы.

    Возвращает:
    - image: ndarray
        Изображение с наложенными bounding box.
    """
    # Добавляем белую рамку
    image_with_border = add_white_border(image)

    # Конвертируем в оттенки серого
    if len(image_with_border.shape) == 3:
        gray = cv2.cvtColor(image_with_border, cv2.COLOR_BGR2GRAY)
    else:
        gray = image_with_border.copy()

    if display_steps:
        cv2_imshow(gray)

    # Обработка изображения
    eroded = preprocess_image(gray)
    eroded = eroded[10:-10, 10:-10]

    if display_steps:
        cv2_imshow(eroded)

    # Поиск контуров
    contours = cv2.findContours(eroded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    # Сегментация и генерация bounding box
    bboxes = []  # Для хранения ограничивающих рамок
    classes = [] # Для хранения класса для каждой рамки (0 -- синий, 1 -- красный)
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        dominant_color = find_dominant_color(image, (x, y, w, h))
        bboxes.append((x, y, x + w, y + h))
        classes.append(0 if dominant_color[0] == 255 else 1)
        cv2.rectangle(image, (x, y), (x + w, y + h), dominant_color, 1)

    # Сохранение аннотаций YOLO
    yolo_annotations = convert_to_yolo_format(image, bboxes, classes)
    save_yolo_annotations(yolo_annotations, filename, yolo_annotations_path)

    if display_steps:
        cv2_imshow(image)

    return image

### Start scr

In [ ]:
def processing(path_to_folder, crop_top, crop_left, crop_right, crop_bottom, output_folder="output"):
    """
    Выполняет обработку изображений из указанной папки: обрезку, обработку контуров
    и сохранение результатов в указанную выходную папку.

    Параметры:
    - path_to_folder: str
        Путь к папке с исходными изображениями.
    - crop_top: float
        Процент обрезки сверху от общей высоты изображения.
    - crop_left: float
        Процент обрезки слева от общей ширины изображения.
    - crop_right: float
        Процент обрезки справа от общей ширины изображения.
    - crop_bottom: float
        Процент обрезки снизу от общей высоты изображения.
    - output_folder: str, optional (default="output")
        Путь к папке, куда будут сохранены обработанные изображения.

    Возвращает:
    - None

    Описание работы:
    1. Загружает изображения из папки `path_to_folder` с использованием `load_images_from_folder`.
    2. Обрезает каждое изображение с помощью `crop_image` на основе заданных процентов обрезки.
    3. Применяет одну из функций обработки:
        - `my_find_contours` для обработки контуров или создания масок.
        - `my_find_contours_with_bbox` для создания bounding box.
    4. Сохраняет обработанные изображения в указанную папку `output_folder`.
    """
    images = load_images_from_folder(folder_path=path_to_folder)

    for i, (name_file, image) in enumerate(images.items()):
        frame = crop_image(image,
                            crop_top=crop_top,
                            crop_left=crop_left,
                            crop_right=crop_right,
                            crop_bottom=crop_bottom)
        # Обработка изображения
        # processed_image = my_find_contours(frame, name_file[:-4], False) # my contours
        processed_image = my_find_contours_with_bbox(frame, name_file[:-4], False) # my bbox

        # Формируем путь для сохранения файла
        output_path = os.path.join(output_folder, f"{name_file}")

        # Сохраняем обработанное изображение
        cv2.imwrite(output_path, processed_image)
        # print(f"Обработанное изображение сохранено: {output_path}")

In [ ]:
crop_top = 0     # %
crop_left = 0    # %
crop_right = 0   # %
crop_bottom = 0  # %

In [ ]:
processing(path_to_folder=path_to_folder,
           crop_top=crop_top,
           crop_right=crop_right,
           crop_left=crop_left,
           crop_bottom=crop_bottom,
           output_folder=output_folder)